In [18]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)

In [21]:
from src.database.connection import connect

In [23]:
with connect(db_key="main") as conn:
    with conn.cursor() as cur:
        pass

TypeError: get_db_url_from_key() got an unexpected keyword argument 'suffix'